# The JPEG standard (ISO/IEC 10918-1)

## Intro

* JPEG = Joint Photographic Experts Group.
* Developed in 1992 by the ISO [[ISO, 1992]](https://idea2ic.com/File_Formats/CCITT.pdf).
* True color (up to 24 bits/pixel) and grayscale (up to 8
  bits/pixel) images.
* Eligible compression quality between $0-100$ [[Wallace, 1992]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=GK+Wallace+jpeg+1992&btnG=).
* Visually lossless reconstructions of color images for
  compression ratios about $1$ bits/pixel (bpp).
* Based on the Discrete Cosine Transform (DCT).
* Several working modes: lossy (JPEG), lossless (JPEG-LS) and hierarchical.

## 1. JPEG lossless [[ISO, 1992]](https://idea2ic.com/File_Formats/CCITT.pdf)

* Based on an spatial predictor and a 0-order static
  variable-length encoder (Huffman).

* The block diagram is:
<img src="JPEG/LS-JPEG_codec.svg" style="width: 400px;"/>
where
<img src="JPEG/contexto_prediccion_JPEG.svg" style="width: 100px;"/>
and
<img src="JPEG/LS-JPEG_predictors.svg" style="width: 200px;"/>

### Coder
1. Generate $\hat{s}$.
2. Compute the prediction error $e\leftarrow s - \hat{s}$.
3. Encode symbol $e$.

### Decoder
1. Generate $\hat{s}$ (idential to the Step 1 of the compressor).
2. Decode symbol $e$.
3. Compute the pixel value $s\leftarrow e+\hat{s}$.

<img src="JPEG/categorias.svg" style="width: 400px;"/>
<img src="JPEG/codigos.svg" style="width: 200px;"/>

### Encode symbol

1. Search $e$ in $DIFF$ and select $SSSS$.
2. Encode $SSSS$ following the base code.
3. If $e>0$, then:
   1. Encode $e$ using a binary number of $SSSS$ bits. The most
      significant bit of $e$ will be always 1.
4. Else:
   2. Encode $e-1$ using a binary number of $SSSS$ bits. The most
    significant bit of $e$ will be always a 0.
    
Example ($e=5$):

* [1] $SSSS=3$.
* [2] Output $\leftarrow 100_2$.
* [3.A] Output $\leftarrow 101_2$.

Example ($e=-9$):

* [1] $SSSS=4$.
* [2] Output $\leftarrow 101_2$.
* [4.A] Output $\leftarrow 0110_2$ (the four least significant bits of the two's complement of $-10_{10}$).

### Decode symbol

1. Decode the $SSSS$ category using the base code.
2. Decode the magnitude. Let $x\leftarrow$ the next input bit.
3. If $x\ne 0$, then:
   1. $e\leftarrow x << (SSSS-1) + \text{next}~(SSSS-1)$ bits.
4. Else:
  1. $e\leftarrow (-1)~\text{AND}~(x << (SSSS-1)+\text{next}~(SSSS-1)~\text{bits}+1)$.
  
Example (decode $100101$):

* [1] $SSSS\leftarrow 3$.
* [2] $x\leftarrow 1$.
* [3.A] $e\leftarrow 2^2+$ next two input bits (01) $=101_2=5_{10}$.

Example (decode $1010110$):

* [1] $SSSS\leftarrow 4$.
* [2] $x\leftarrow 0$.
* [4.A] (Using 8 bits of precision) $e\leftarrow$ $11111111_2$ AND ($0000_2+110_2+1_2) = 11110111_2 = -9_{10}$ (the four least significant bits of $0111_2$ are supposed to be $1$.

In [ ]:
!wget https://sourceforge.net/projects/jpeg/files/jpeg/Cornell%20LJPEG%20v1.0/ljpg.tar.Z/download?use_mirror=ayera&download=&failedmirror=kent.dl.sourceforge.net
!uncompress ljpg.tar.Z
!tar xv ljpg.tar
%cd ljpg
!make